In [2]:
import sys; sys.path.append('scripts')
from common import *
from global_common import *
from models import *
import xgboost as xgb
from load_and_proccess_data import *
from visualization import *

In [3]:
train_df, test_df, columns_type_dict = load_data()

Из целочисленныъ - 4 бинарных.
Целочисленных переменных  : 232
Переменных с плавающей точкой : 225
Переменным присвоен соответствующий тип.


In [4]:
one_value_cols = ['rr_act_type_7', 'rr_gift_type_1', 'voice_omo_in_cost_m1', 'voice_pstn_in_cost_m1', 
                  'voice_omo_in_cost_m2', 'voice_pstn_in_cost_m2', 'voice_omo_in_cost_m3', 'voice_pstn_in_cost_m3']

to_drop = ['inact_days_count', 'rr_gift_type_2', 'count_act_type_5', 'count_act_type_7', 
           'count_gift_type_2', 'paym_sum_m1', 'voice_onnet_out_dur_m1', 'voice_onnet_in_dur_m1', 
           'voice_pstn_out_dur_m1', 'voice_onnet_out_count_m1', 'voice_onnet_in_count_m1', 'sms_roam_in_count_m1', 
           'service_8_count_m1', 'paym_sum_m2', 'voice_onnet_out_dur_m2', 'voice_onnet_in_dur_m2', 
           'voice_onnet_out_count_m2', 'voice_onnet_in_count_m2', 'sms_roam_in_count_m2', 'service_8_count_m2', 
           'voice_onnet_out_dur_m3', 'voice_onnet_in_dur_m3', 'voice_onnet_out_count_m3', 'voice_onnet_in_count_m3', 
           'voice_omo_cc_count_m3', 'sms_roam_in_count_m3', 'service_8_count_m3'] + \
           ['sms_roam_in_count_m3'] + one_value_cols

train_df = train_df.drop(columns = to_drop)
test_df = test_df.drop(columns = to_drop)

In [5]:
dep_var = 'target'
cont_names = train_df.select_dtypes('float').columns
cat_names = ['os_category', 'device_type', 'service_7_flag_m1', 
             'service_7_flag_m2', 'service_7_flag_m3', 'manufacturer_category']
procs = [Categorify, Normalize] 

In [6]:
path_to_data = '../data'

In [7]:
roc_auc = RocAuc(roc_auc_score)

In [8]:
test = TabularList.from_df(test_df.drop('id', axis=1), path=path_to_data, cat_names=cat_names, cont_names=cont_names)

In [9]:
# Здесь нужно получить только индексы для валидионной выборки мы их передадим в фастаи для разделения выборок
kf = 5
val_folds_idx = []
skf = StratifiedKFold(n_splits=kf, random_state=SEED, shuffle=True)
for train_idx, valid_idx in skf.split(train_df.index, train_df.target):
    val_folds_idx.append(valid_idx)

In [12]:
check = []
# Гоняем kf моделей и сохраняем лучшие
predictions = np.zeros((test_df.shape[0]))
for fold in range(kf):
    # Создаем данные для подвыборки
    data = (TabularList.from_df(train_df.drop('id', axis=1), path=path_to_data, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(val_folds_idx[fold])
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())
    data.batch_size = 64
    # Создаем модель 
    learn = tabular_learner(data, 
                            layers=[300,200], 
                            metrics=[accuracy, roc_auc], 
                            emb_drop=0.01, 
                            ps=0.5,
                           )
    
    # Считаем
    learn.fit_one_cycle(15, max_lr=1e-2)
    
    # Считаем сразу предикшн
    p = learn.get_preds(ds_type=DatasetType.Test)
    p_list = p[0][:, 1].tolist()
    check.append(p_list)
    predictions += p_list
    
    del data
    del learn
    
# Получаем средний предикшн
predictions /= kf

epoch,train_loss,valid_loss,accuracy,roc_auc_score
1,0.243921,0.250165,0.923363,0.709326
2,0.248179,0.246855,0.923863,0.729624
3,0.238161,0.246874,0.924077,0.736848
4,0.240576,36.924698,0.919863,0.751127
5,0.240705,0.307528,0.922506,0.744383
6,0.236256,0.846746,0.923363,0.756340
7,0.235964,0.300719,0.923505,0.752128
8,0.238341,0.325620,0.921648,0.755915
9,0.239606,0.293811,0.923434,0.753493
10,0.237802,108.863495,0.921291,0.759064


KeyboardInterrupt: 

In [ ]:
make_submition(test_df, learn, 'local_rocauc_83')